In [15]:
import pandas as pd
data=pd.read_csv('abcnews-date-text.csv')
data_text = data[['headline_text']]
data_text['index'] = data_text.index
documents = data_text

In [16]:
print(len(documents))
print(documents[:5])

1103663
                                       headline_text  index
0  aba decides against community broadcasting lic...      0
1     act fire witnesses must be aware of defamation      1
2     a g calls for infrastructure protection summit      2
3           air nz staff in aust strike for pay rise      3
4      air nz strike to affect australian travellers      4


In [17]:
#!pip install gensim

In [18]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(2018)
import nltk
#nltk.download('wordnet')

In [19]:
def lemmatize_stemming(text):
    stemmer = PorterStemmer()
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))

In [20]:
def preprocess(text):
    result=[]
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token)>3:
            result.append(lemmatize_stemming(token))
    return result

In [21]:
doc_sample=documents[documents['index']==4310].values[0][0]
print('original document: ')
words = []
for word in doc_sample.split(" "):
    words.append(word)
print(words)
print('\n\n tokenized and lemmatized document: ')
print(preprocess(doc_sample))

original document: 
['rain', 'helps', 'dampen', 'bushfires']


 tokenized and lemmatized document: 
['rain', 'help', 'dampen', 'bushfir']


In [22]:
processed_docs = documents['headline_text'].map(preprocess)
processed_docs[:10]

0               [decid, commun, broadcast, licenc]
1                               [wit, awar, defam]
2           [call, infrastructur, protect, summit]
3                      [staff, aust, strike, rise]
4             [strike, affect, australian, travel]
5               [ambiti, olsson, win, tripl, jump]
6           [antic, delight, record, break, barca]
7    [aussi, qualifi, stosur, wast, memphi, match]
8            [aust, address, secur, council, iraq]
9                         [australia, lock, timet]
Name: headline_text, dtype: object

In [23]:
dictionary = gensim.corpora.Dictionary(processed_docs)
count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break

0 broadcast
1 commun
2 decid
3 licenc
4 awar
5 defam
6 wit
7 call
8 infrastructur
9 protect
10 summit


In [24]:
dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)

In [25]:
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]
bow_corpus[4310]

[(76, 1), (112, 1), (484, 1), (4022, 1)]

In [26]:
#to check bow
bow_doc_4310 = bow_corpus[4310]
for i in range(len(bow_doc_4310)):
    print("Word {} (\"{}\") appears {} time.".format(bow_doc_4310[i][0], 
                                               dictionary[bow_doc_4310[i][0]], 
bow_doc_4310[i][1]))

Word 76 ("bushfir") appears 1 time.
Word 112 ("help") appears 1 time.
Word 484 ("rain") appears 1 time.
Word 4022 ("dampen") appears 1 time.


In [27]:
from gensim import corpora, models
tfidf = models.TfidfModel(bow_corpus)
corpus_tfidf = tfidf[bow_corpus]

In [28]:
from pprint import pprint
for doc in corpus_tfidf:
    pprint(doc)
    break

[(0, 0.5903873663261261),
 (1, 0.38522586675616677),
 (2, 0.49747840165121515),
 (3, 0.5055284652272439)]


In [29]:
## Running LDA using Bag OF Words
lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=10, id2word=dictionary, passes=2, workers=2)

In [30]:
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.025*"queensland" + 0.021*"adelaid" + 0.018*"market" + 0.016*"south" + 0.012*"share" + 0.012*"farmer" + 0.011*"victoria" + 0.011*"busi" + 0.010*"flood" + 0.010*"water"
Topic: 1 
Words: 0.056*"australia" + 0.035*"trump" + 0.026*"world" + 0.023*"nation" + 0.016*"tasmania" + 0.013*"island" + 0.011*"polit" + 0.010*"take" + 0.009*"hobart" + 0.009*"drum"
Topic: 2 
Words: 0.024*"plan" + 0.018*"council" + 0.016*"health" + 0.016*"rural" + 0.014*"indigen" + 0.013*"turnbul" + 0.013*"commun" + 0.013*"chang" + 0.013*"servic" + 0.011*"need"
Topic: 3 
Words: 0.025*"kill" + 0.022*"attack" + 0.019*"famili" + 0.018*"donald" + 0.015*"fight" + 0.014*"dead" + 0.013*"crash" + 0.012*"vote" + 0.011*"victim" + 0.011*"senat"
Topic: 4 
Words: 0.019*"melbourn" + 0.019*"countri" + 0.019*"test" + 0.018*"live" + 0.014*"gold" + 0.013*"life" + 0.013*"feder" + 0.013*"women" + 0.012*"student" + 0.012*"tasmanian"
Topic: 5 
Words: 0.030*"elect" + 0.022*"canberra" + 0.018*"say" + 0.013*"worker" + 0.011*"r

In [31]:
## Running LDA using TF-IDF
lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, num_topics=10, id2word=dictionary, passes=2, workers=4)
for idx, topic in lda_model_tfidf.print_topics(-1):
    print('Topic: {} Word: {}'.format(idx, topic))

Topic: 0 Word: 0.017*"polic" + 0.016*"charg" + 0.015*"murder" + 0.012*"woman" + 0.012*"court" + 0.011*"death" + 0.009*"jail" + 0.009*"shoot" + 0.008*"assault" + 0.008*"search"
Topic: 1 Word: 0.012*"govern" + 0.009*"health" + 0.008*"fund" + 0.006*"budget" + 0.006*"royal" + 0.006*"servic" + 0.006*"busi" + 0.006*"minist" + 0.006*"commiss" + 0.005*"christma"
Topic: 2 Word: 0.015*"rural" + 0.014*"south" + 0.013*"coast" + 0.012*"north" + 0.012*"west" + 0.010*"gold" + 0.010*"donald" + 0.007*"news" + 0.007*"marriag" + 0.007*"korea"
Topic: 3 Word: 0.008*"monday" + 0.007*"abbott" + 0.006*"terror" + 0.006*"polic" + 0.006*"crop" + 0.005*"drive" + 0.005*"social" + 0.005*"dairi" + 0.005*"onlin" + 0.005*"grain"
Topic: 4 Word: 0.010*"street" + 0.009*"michael" + 0.008*"octob" + 0.008*"cyclon" + 0.008*"wall" + 0.007*"teenag" + 0.006*"wild" + 0.006*"anim" + 0.006*"human" + 0.005*"smith"
Topic: 5 Word: 0.011*"leagu" + 0.010*"turnbul" + 0.009*"final" + 0.007*"world" + 0.007*"rugbi" + 0.006*"decemb" + 0.006

In [32]:
#testing
unseen_document = 'There was a attack in Mumbai'
bow_vector = dictionary.doc2bow(preprocess(unseen_document))
for index, score in sorted(lda_model[bow_vector], key=lambda tup: -1*tup[1]):
    print("Score: {}\t Topic: {}".format(score, lda_model.print_topic(index, 5)))

Score: 0.6999134421348572	 Topic: 0.025*"kill" + 0.022*"attack" + 0.019*"famili" + 0.018*"donald" + 0.015*"fight"
Score: 0.03334446996450424	 Topic: 0.054*"polic" + 0.020*"perth" + 0.018*"die" + 0.017*"interview" + 0.016*"miss"
Score: 0.033342763781547546	 Topic: 0.025*"queensland" + 0.021*"adelaid" + 0.018*"market" + 0.016*"south" + 0.012*"share"
Score: 0.033342763781547546	 Topic: 0.056*"australia" + 0.035*"trump" + 0.026*"world" + 0.023*"nation" + 0.016*"tasmania"
Score: 0.033342763781547546	 Topic: 0.024*"plan" + 0.018*"council" + 0.016*"health" + 0.016*"rural" + 0.014*"indigen"
Score: 0.033342763781547546	 Topic: 0.019*"melbourn" + 0.019*"countri" + 0.019*"test" + 0.018*"live" + 0.014*"gold"
Score: 0.033342763781547546	 Topic: 0.030*"elect" + 0.022*"canberra" + 0.018*"say" + 0.013*"worker" + 0.011*"royal"
Score: 0.033342763781547546	 Topic: 0.020*"state" + 0.018*"hour" + 0.017*"rise" + 0.015*"call" + 0.015*"govern"
Score: 0.033342763781547546	 Topic: 0.035*"charg" + 0.033*"court" 